### Проверка работы модели на тестовых данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, classification_report
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [2]:
TEST_DATASET_PATH = '/Users/annasmejoukha/Desktop/ML_Business/project/datasets/test.csv'
test_df = pd.read_csv(TEST_DATASET_PATH, index_col=0)
test_df.head(2)

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,...,4,4,4,4,3,4,5,0,0.0,satisfied


In [3]:
test_df.shape

(25976, 24)

Проверка модели на тестовых данных
Разбиваем датасет на X и y

In [4]:
X = test_df.drop('satisfaction', axis=1)
y = pd.DataFrame({'satisfaction': test_df['satisfaction']})

Заменяем значения целевого признака на 1 и 0 для последущего расчета метрик качества модели

In [5]:
y.satisfaction = np.where(y.satisfaction == 'satisfied', 1, 0)

Загрузим модель из файла и делаем предсказания

In [6]:
with open('/Users/annasmejoukha/Desktop/ML_Business/project/models/xgbclf_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [7]:
predictions = pipeline.predict(X)
pd.DataFrame({'preds': predictions}).to_csv("/Users/annasmejoukha/Desktop/ML_Business/project/datasets/test_predictions.csv", index=None)

In [8]:
binary_preds = np.where(predictions == 'satisfied', 1, 0)

In [9]:
print(f'ROC AUC score: {roc_auc_score(y, binary_preds)}')

ROC AUC score: 0.961491592533332


In [10]:
print(f'Classification report:\n{classification_report(y, binary_preds)}')

Classification report:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     14573
           1       0.97      0.94      0.96     11403

    accuracy                           0.96     25976
   macro avg       0.97      0.96      0.96     25976
weighted avg       0.96      0.96      0.96     25976

